In [20]:
import torch
from glob import glob
from ast import literal_eval
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List
from IPython.core.display import display, HTML
from sklearn.manifold import TSNE
from tqdm.notebook import tqdm

In [21]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)

plt.style.use('dark_background')

display(HTML("<style>"
    + "#notebook { padding-top:0px; } " ""
    + ".container { width:100%; } "
    + ".end_space { min-height:0px; } "
    + "</style>"))

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cuda


In [22]:
def get_name_exp(exp: dict):
    interesting_props = ['con_weight', 'spectral_buffer_size']
    props = {}
    for prop in interesting_props:
        if prop in exp:
            props[prop] = exp[prop]
    if len(props) == 0: props = ''
    return f'{exp["model"]} {props}'

def print_exp_info(exp: dict):
    print(f'model: {get_name_exp(exp)}')
    print({k: exp[k] for k in exp if k not in ['latents']})

In [23]:
def flatten_list(l: List[List[any]]) -> List[any]:
    return [item for line in l for item in line]

In [24]:
# logs_path = 'C:\\Users\\emace\\AImageLab\\SRV-Continual\\results\\canomaly\\logs'
logs_path = '/nas/softechict-nas-2/efrascaroli/mammoth-data/logs'
# logs_path = '/nas/softechict-nas-1/rbenaglia/canomaly-data/logs'
# logs_path = '../storage/logs'

In [25]:
exp_list = []
environments = []
for log_file in glob(logs_path + '/**/*ace*/*res*.pyd', recursive=True):
    print(log_file)
    with open(log_file, 'r') as f:
        props_list = [prop for prop in log_file.replace(logs_path, '').split('/')[1:]]
        print(f'Env: {props_list}')
        props = props_list
        exps = []
        for i, line in enumerate(f.readlines()):
            try:
                exps.append(literal_eval(line.replace('nan', 'None')))
            except:
                print(f'Unparsed line {i}:\n\t{exps[:-1]}\n-->\t{line}')

        environments.append({'env': props, 'exps': exps})

        exp_list.extend(exps)

/nas/softechict-nas-2/efrascaroli/mammoth-data/logs/seq-cifar10/er_ace_con/results.pyd
Env: ['seq-cifar10', 'er_ace_con', 'results.pyd']
/nas/softechict-nas-2/efrascaroli/mammoth-data/logs/seq-cifar10/er_ace_con/res1.pyd
Env: ['seq-cifar10', 'er_ace_con', 'res1.pyd']
/nas/softechict-nas-2/efrascaroli/mammoth-data/logs/seq-cifar10/er_ace_con_pre/results.pyd
Env: ['seq-cifar10', 'er_ace_con_pre', 'results.pyd']
/nas/softechict-nas-2/efrascaroli/mammoth-data/logs/seq-cifar10/er_ace_con_pre/res100.pyd
Env: ['seq-cifar10', 'er_ace_con_pre', 'res100.pyd']
/nas/softechict-nas-2/efrascaroli/mammoth-data/logs/seq-cifar100-10x10/er_ace_con/res1.pyd
Env: ['seq-cifar100-10x10', 'er_ace_con', 'res1.pyd']


In [26]:
print(f'Total exps: {len(exp_list)}')
# exp_list[0]['results']

Total exps: 156


In [27]:
env = environments[0]
print(env['env'])
discriminant = 'con_weight'
important_task = [2, 3, 4]
results_dict = {}
for idx, exp in enumerate(env['exps']):
    dis = exp[discriminant]
    if dis not in results_dict:
        results_dict[dis] = {'n_exps': 0, 'exp_indexes': []}
        for i in important_task:
            results_dict[dis][f'CIL-{i}'] = []
            results_dict[dis][f'TIL-{i}'] = []
            results_dict[dis][f'ConE-{i}'] = []
    results_dict[dis]['n_exps'] += 1
    results_dict[dis]['exp_indexes'].append(idx)
    for i in important_task:
        results_dict[dis][f'CIL-{i}'].append(exp['results'][i-1]['Class-IL mean'])
        results_dict[dis][f'TIL-{i}'].append(exp['results'][i-1]['Task-IL mean'])
        results_dict[dis][f'ConE-{i}'].append(exp['results'][i-1]['Con-Error'])

['seq-cifar10', 'er_ace_con', 'results.pyd']


In [28]:
df = pd.DataFrame(index=[f'Task {i}' for i in important_task],
                  columns=['Class-IL', 'Task-IL', 'Con-Error'])
for dis in results_dict:
    res = results_dict[dis]
    print(f'con weight: {dis} | exps({res["n_exps"]}): {res["exp_indexes"]}\n')
    for i in important_task:
        df.loc[f'Task {i}', 'Class-IL']     = f'{np.mean(res[f"CIL-{i}"]):.2f} ± {np.std(res[f"CIL-{i}"]):.2f}'
        df.loc[f'Task {i}', 'Task-IL']      = f'{np.mean(res[f"TIL-{i}"]):.2f} ± {np.std(res[f"TIL-{i}"]):.2f}'
        df.loc[f'Task {i}', 'Con-Error']    = f'{np.mean(res[f"ConE-{i}"]):.2f} ± {np.std(res[f"ConE-{i}"]):.2f}'
    display(df)

con weight: 0.1 | exps(5): [0, 7, 11, 16, 18]



,Class-IL,Task-IL,Con-Error
Task 2,58.10 ± 4.70,73.22 ± 4.19,24.26 ± 1.00
Task 3,47.71 ± 3.33,76.72 ± 3.34,23.16 ± 0.43
Task 4,41.77 ± 3.39,77.02 ± 2.31,21.47 ± 0.67


con weight: 0.001 | exps(5): [1, 4, 8, 12, 15]



,Class-IL,Task-IL,Con-Error
Task 2,79.90 ± 2.82,91.24 ± 0.98,23.96 ± 0.86
Task 3,64.96 ± 2.70,90.24 ± 0.86,22.56 ± 0.78
Task 4,60.64 ± 3.47,89.52 ± 1.66,21.29 ± 0.74


con weight: 0.01 | exps(5): [2, 5, 9, 13, 17]



,Class-IL,Task-IL,Con-Error
Task 2,80.25 ± 1.51,91.02 ± 0.75,22.78 ± 1.85
Task 3,65.81 ± 2.81,90.42 ± 1.18,21.67 ± 0.61
Task 4,61.70 ± 2.01,90.90 ± 0.67,20.75 ± 0.56


con weight: 0.0 | exps(5): [3, 6, 10, 14, 19]



,Class-IL,Task-IL,Con-Error
Task 2,82.32 ± 1.38,91.92 ± 0.73,26.63 ± 0.95
Task 3,66.85 ± 1.95,90.08 ± 0.91,25.58 ± 0.99
Task 4,61.18 ± 1.39,90.22 ± 0.82,23.91 ± 1.21


In [29]:
env = environments[4]
print(env['env'])
discriminant = 'con_weight'
important_task = [2]
divide_accs = [1, 2]
results_dict = {}
for idx, exp in enumerate(env['exps']):
    dis = exp[discriminant]
    if dis not in results_dict:
        results_dict[dis] = {'n_exps': 0, 'exp_indexes': [], 'Task-1': [], }
        for i in divide_accs:
            results_dict[dis][f'CIL-{i}'] = []
            results_dict[dis][f'TIL-{i}'] = []
            results_dict[dis][f'ConE-{i}'] = []
    results_dict[dis]['n_exps'] += 1
    results_dict[dis]['exp_indexes'].append(idx)
    results_dict[dis]['Task-1'].append(exp['results'][0]['Class-IL'][0])
    for i in important_task:
        for j in divide_accs:
            results_dict[dis][f'CIL-{j}'].append(exp['results'][i-1]['Class-IL'][j-1])
            results_dict[dis][f'TIL-{j}'].append(exp['results'][i-1]['Task-IL'][j-1])
            results_dict[dis][f'ConE-{j}'].append(exp['results'][i-1]['Con-Error'])

['seq-cifar100-10x10', 'er_ace_con', 'res1.pyd']


In [30]:
df = pd.DataFrame(index=[f'Task {i}' for i in divide_accs],
                  columns=['Class-IL', 'Task-IL', 'Con-Error'])
for dis in results_dict:
    res = results_dict[dis]
    print(f'con weight: {dis} | exps({res["n_exps"]}): {res["exp_indexes"]}\n'
          f'Task-1: {np.mean(res[f"Task-1"]):.2f} ± {np.std(res[f"Task-1"]):.2f}')
    for i in divide_accs:
        df.loc[f'Task {i}', 'Class-IL']     = f'{np.mean(res[f"CIL-{i}"]):.2f} ± {np.std(res[f"CIL-{i}"]):.2f}'
        df.loc[f'Task {i}', 'Task-IL']      = f'{np.mean(res[f"TIL-{i}"]):.2f} ± {np.std(res[f"TIL-{i}"]):.2f}'
        df.loc[f'Task {i}', 'Con-Error']    = f'{np.mean(res[f"ConE-{i}"]):.2f} ± {np.std(res[f"ConE-{i}"]):.2f}'
    display(df)

con weight: 0.0 | exps(6): [0, 1, 6, 7, 10, 11]
Task-1: 76.97 ± 10.08


,Class-IL,Task-IL,Con-Error
Task 1,50.53 ± 3.11,67.98 ± 3.20,26.04 ± 1.40
Task 2,70.90 ± 1.38,84.53 ± 1.53,26.04 ± 1.40


con weight: 0.001 | exps(5): [2, 4, 8, 12, 15]
Task-1: 80.96 ± 3.42


,Class-IL,Task-IL,Con-Error
Task 1,50.68 ± 6.49,69.70 ± 2.09,17.75 ± 0.92
Task 2,68.76 ± 4.33,83.86 ± 2.83,17.75 ± 0.92


con weight: 0.01 | exps(5): [3, 5, 9, 13, 14]
Task-1: 74.18 ± 8.31


,Class-IL,Task-IL,Con-Error
Task 1,53.60 ± 2.64,70.66 ± 3.05,16.63 ± 1.74
Task 2,68.78 ± 4.55,84.12 ± 1.27,16.63 ± 1.74


In [31]:

env = environments[3]
print(env['env'])
important_task = [2, 3, 4, 5]
divide_accs = [1, 2]
results_dict = {}
for idx, exp in enumerate(env['exps']):
    dis = get_name_exp(exp)
    if dis not in results_dict:
        results_dict[dis] = {'n_exps': 0, 'exp_indexes': []}
        for i in important_task:
            results_dict[dis][f'CIL-{i}'] = []
            results_dict[dis][f'TIL-{i}'] = []
            results_dict[dis][f'ConE-{i}'] = []
            results_dict[dis][f'C100-{i}'] = []
    results_dict[dis]['n_exps'] += 1
    results_dict[dis]['exp_indexes'].append(idx)
    for i in important_task:
        results_dict[dis][f'CIL-{i}'].append(exp['results'][i-1]['Class-IL mean'])
        results_dict[dis][f'TIL-{i}'].append(exp['results'][i-1]['Task-IL mean'])
        results_dict[dis][f'ConE-{i}'].append(exp['results'][i-1]['Con-Error'])
        results_dict[dis][f'C100-{i}'].append(exp['results'][i-1]['Cifar100-acc'] * 100)

['seq-cifar10', 'er_ace_con_pre', 'res100.pyd']


In [32]:
df = pd.DataFrame(index=[f'Task {i}' for i in important_task],
                  columns=['Class-IL', 'Task-IL', 'Con-Error', 'C100'])
for dis in results_dict:
    res = results_dict[dis]
    print(f'| {dis} | exps({res["n_exps"]}): {res["exp_indexes"]}\n')
    for i in important_task:
        df.loc[f'Task {i}', 'Class-IL']     = f'{np.mean(res[f"CIL-{i}"]):.2f} ± {np.std(res[f"CIL-{i}"]):.2f}'
        df.loc[f'Task {i}', 'Task-IL']      = f'{np.mean(res[f"TIL-{i}"]):.2f} ± {np.std(res[f"TIL-{i}"]):.2f}'
        df.loc[f'Task {i}', 'Con-Error']    = f'{np.mean(res[f"ConE-{i}"]):.2f} ± {np.std(res[f"ConE-{i}"]):.2f}'
        df.loc[f'Task {i}', 'C100']         = f'{np.mean(res[f"C100-{i}"]):.2f} ± {np.std(res[f"C100-{i}"]):.2f}'
    display(df)

| er_ace_con_pre {'con_weight': 0.0, 'spectral_buffer_size': 100} | exps(5): [0, 4, 9, 12, 16]



,Class-IL,Task-IL,Con-Error,C100
Task 2,89.32 ± 2.05,96.08 ± 1.39,25.91 ± 0.47,59.29 ± 0.50
Task 3,77.84 ± 1.06,95.33 ± 0.58,28.02 ± 0.16,55.06 ± 0.47
Task 4,74.40 ± 1.45,95.68 ± 0.52,27.81 ± 0.30,54.27 ± 0.43
Task 5,67.92 ± 1.25,94.97 ± 0.47,26.56 ± 0.35,51.24 ± 0.53


| er_ace_con_pre {'con_weight': 0.01, 'spectral_buffer_size': 100} | exps(5): [1, 6, 10, 14, 17]



,Class-IL,Task-IL,Con-Error,C100
Task 2,89.42 ± 1.14,96.34 ± 0.26,16.99 ± 1.75,59.30 ± 0.51
Task 3,78.90 ± 0.35,95.45 ± 0.12,19.70 ± 1.64,55.93 ± 0.57
Task 4,75.09 ± 0.46,95.82 ± 0.44,20.32 ± 1.39,55.00 ± 0.83
Task 5,68.64 ± 0.71,94.87 ± 0.59,20.54 ± 0.86,52.09 ± 0.57


| er_ace_con_pre {'con_weight': 0.1, 'spectral_buffer_size': 100} | exps(5): [2, 5, 8, 13, 18]



...

,Class-IL,Task-IL,Con-Error,C100
Task 2,89.79 ± 0.59,96.30 ± 0.32,19.31 ± 1.11,59.41 ± 0.31
Task 3,79.02 ± 0.89,95.44 ± 0.22,21.57 ± 0.63,56.11 ± 0.30
Task 4,76.22 ± 0.51,95.66 ± 0.28,21.99 ± 0.67,54.92 ± 0.74
Task 5,67.33 ± 0.67,94.81 ± 0.45,22.33 ± 0.68,52.10 ± 0.64


| er_ace_con_pre {'con_weight': 0.01, 'spectral_buffer_size': 1000} | exps(3): [20, 22, 24]



,Class-IL,Task-IL,Con-Error,C100
Task 2,90.86 ± 1.16,96.63 ± 0.53,7.45 ± 0.61,61.01 ± 0.24
Task 3,79.07 ± 0.40,95.30 ± 0.20,9.78 ± 1.37,58.84 ± 0.36
Task 4,75.59 ± 1.08,95.84 ± 0.24,12.04 ± 2.11,58.24 ± 0.34
Task 5,71.02 ± 0.34,95.31 ± 0.14,13.47 ± 1.69,56.43 ± 0.32


| er_ace_con_pre {'con_weight': 0.001, 'spectral_buffer_size': 1000} | exps(3): [21, 23, 25]



,Class-IL,Task-IL,Con-Error,C100
Task 2,90.40 ± 0.34,96.79 ± 0.33,8.73 ± 0.39,60.20 ± 0.08
Task 3,79.83 ± 0.25,95.78 ± 0.22,11.55 ± 0.23,58.15 ± 0.64
Task 4,77.22 ± 0.71,96.31 ± 0.22,12.70 ± 0.64,58.72 ± 0.27
Task 5,70.33 ± 0.66,95.53 ± 0.54,13.89 ± 0.70,55.24 ± 0.53
